In [1]:
import pandas as pd
import chess
from tqdm import tqdm
import re


df = pd.read_csv('../data/raw/df_chess_inputs.csv')

In [2]:
df.head(2)

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,evals,ECO_Family
0,Eduardo,Clase,1,200,1200,?,40/9000:40/9000:40/9000,normal,1. e4 {0.00/1 1} b5 {(Bf1xb5 Bc8-b7) -1.79/1 1...,"[0.34, -0.24, 1.92, -1.73, 1.83, -1.52, 2.37, ...",?


In [3]:
df['AN'][0]

'1. e4 {0.00/1 1} b5 {(Bf1xb5 Bc8-b7) -1.79/1 1} 2. Bxb5 {(Bc8-b7 d2-d3)\n+1.79/1 1} e6 {(Nb1-c3) -1.79/1 1} 3. Nf3 {(Ng8-f6) +1.74/1 1} c6 {(Bb5-e2\nNg8-f6) -1.93/1 1} 4. Ba4 d5 {(d2-d3) -2.16/1 1} 5. exd5 {(e6xd5 O-O)\n+2.28/1 1} Nf6 {(d5xc6) -3.92/1 1} 6. dxe6 {(Bc8xe6 d2-d4) +3.18/1 1} Bb4\n{(e6xf7+ Ke8-f8) -4.54/1 1} 7. exf7+ {(Ke8-f8 Nb1-c3) +4.18/1 1} Kxf7\n{(Ba4-b3+ Nf6-d5) -4.89/1 1} 8. Ne5+ {(Kf7-e8 Ne5xc6) +3.98/1 1} Kf8 {(O-O\nc6-c5) -6.09/1 1} 9. Qf3 {(Qd8-e7) +2.79/1 1} Bg4 {(Ne5xg4 Qd8-e8+ Ng4-e3)\n-5.75/1 1} 10. Nxg4 {(Qd8-e8+ Ng4-e3) +5.75/1 1} Na6 {(O-O Na6-c5) -6.70/1\n1} 11. O-O {(Na6-c5 Ng4xf6 Nc5xa4) +6.70/1 1} Ke7 {(Ng4xf6 g7xf6) -7.28/1\n1} 12. Nxf6 {(Bb4-d6 Ba4xc6) +7.14/1 1} gxf6 {(Qf3-e4+ Ke7-d6 Qe4xc6+\nKd6-e5) -8.74/1 1} 13. Re1+ {(Ke7-f7 Ba4xc6) +7.08/1 1} Kd7 {(Qf3xc6+)\n-M1/1 1} 14. Qxc6# 1-0'

In [4]:

def extraer_jugadas_san(anotacion):
    # Eliminar las partes entre llaves que contienen las valoraciones y movimientos adicionales
    limpio = re.sub(r'\{[^}]*\}', '', anotacion)
    
    # Eliminar los números de turno (1., 2., etc.)
    limpio = re.sub(r'\d+\.', '', limpio)
    
    # Eliminar los espacios innecesarios y las cadenas como "1-0", "0-1", "1/2-1/2"
    limpio = re.sub(r'\s+', ' ', limpio).strip()
    limpio = re.sub(r'\s*(1-0|0-1|1/2-1/2)\s*$', '', limpio)
    
    # Separar la cadena por espacios
    jugadas = limpio.split(' ')
    
    return jugadas

# Ejemplo de uso
ejemplo = df['AN'][0]
jugadas = extraer_jugadas_san(ejemplo)

print(jugadas[:10])


['e4', 'b5', 'Bxb5', 'e6', 'Nf3', 'c6', 'Ba4', 'd5', 'exd5', 'Nf6']


In [5]:
df.head()

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,evals,ECO_Family
0,Eduardo,Clase,1,200,1200,?,40/9000:40/9000:40/9000,normal,1. e4 {0.00/1 1} b5 {(Bf1xb5 Bc8-b7) -1.79/1 1...,"[0.34, -0.24, 1.92, -1.73, 1.83, -1.52, 2.37, ...",?


In [6]:
from tqdm import tqdm

def extraer_jugadas_san(anotacion):
    # Eliminar las partes entre llaves que contienen las valoraciones y movimientos adicionales
    limpio = re.sub(r'\{[^}]*\}', '', anotacion)
    
    # Eliminar los números de turno (1., 2., etc.)
    limpio = re.sub(r'\d+\.', '', limpio)
    
    # Eliminar los espacios innecesarios y las cadenas como "1-0", "0-1", "1/2-1/2"
    limpio = re.sub(r'\s+', ' ', limpio).strip()
    limpio = re.sub(r'\s*(1-0|0-1|1/2-1/2)\s*$', '', limpio)
    
    # Separar la cadena por espacios
    jugadas = limpio.split(' ')
    
    return jugadas


tqdm.pandas()  
df['SAN'] = df['AN'].progress_apply(extraer_jugadas_san)

# Ver ejemplo
print(df[['AN', 'SAN']].head(1))

100%|██████████| 1/1 [00:00<00:00, 999.36it/s]

                                                  AN  \
0  1. e4 {0.00/1 1} b5 {(Bf1xb5 Bc8-b7) -1.79/1 1...   

                                                 SAN  
0  [e4, b5, Bxb5, e6, Nf3, c6, Ba4, d5, exd5, Nf6...  


In [7]:
df = df.drop(['AN'],axis=1)
df.head()

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,evals,ECO_Family,SAN
0,Eduardo,Clase,1,200,1200,?,40/9000:40/9000:40/9000,normal,"[0.34, -0.24, 1.92, -1.73, 1.83, -1.52, 2.37, ...",?,"[e4, b5, Bxb5, e6, Nf3, c6, Ba4, d5, exd5, Nf6..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   White        1 non-null      object
 1   Black        1 non-null      object
 2   Result       1 non-null      int64 
 3   WhiteElo     1 non-null      int64 
 4   BlackElo     1 non-null      int64 
 5   ECO          1 non-null      object
 6   TimeControl  1 non-null      object
 7   Termination  1 non-null      object
 8   evals        1 non-null      object
 9   ECO_Family   1 non-null      object
 10  SAN          1 non-null      object
dtypes: int64(3), object(8)
memory usage: 220.0+ bytes


In [9]:
'''import ast

# Función para convertir cadenas de texto que representan listas en listas de valores flotantes
def convertir_evals_a_float(eval_str):
    if isinstance(eval_str, str):
        try:
            # Convertir la cadena que representa una lista en una lista real
            eval_list = ast.literal_eval(eval_str)
            if isinstance(eval_list, list):
                return [float(i) for i in eval_list]  # Convertir cada elemento de la lista a float
        except (ValueError, SyntaxError):
            pass  # Si no se puede convertir, simplemente lo devolvemos tal cual
    return eval_str  # Si no es una cadena, devolvemos el valor tal cual

# Aplicar la conversión a la columna 'evals'
df['evals'] = df['evals'].apply(convertir_evals_a_float)

# Ver el resultado
print(df[['evals']])
'''

"import ast\n\n# Función para convertir cadenas de texto que representan listas en listas de valores flotantes\ndef convertir_evals_a_float(eval_str):\n    if isinstance(eval_str, str):\n        try:\n            # Convertir la cadena que representa una lista en una lista real\n            eval_list = ast.literal_eval(eval_str)\n            if isinstance(eval_list, list):\n                return [float(i) for i in eval_list]  # Convertir cada elemento de la lista a float\n        except (ValueError, SyntaxError):\n            pass  # Si no se puede convertir, simplemente lo devolvemos tal cual\n    return eval_str  # Si no es una cadena, devolvemos el valor tal cual\n\n# Aplicar la conversión a la columna 'evals'\ndf['evals'] = df['evals'].apply(convertir_evals_a_float)\n\n# Ver el resultado\nprint(df[['evals']])\n"

In [10]:
import ast

# Asegurarse de que evals es una lista real, no string
df['evals'] = df['evals'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   White        1 non-null      object
 1   Black        1 non-null      object
 2   Result       1 non-null      int64 
 3   WhiteElo     1 non-null      int64 
 4   BlackElo     1 non-null      int64 
 5   ECO          1 non-null      object
 6   TimeControl  1 non-null      object
 7   Termination  1 non-null      object
 8   evals        1 non-null      object
 9   ECO_Family   1 non-null      object
 10  SAN          1 non-null      object
dtypes: int64(3), object(8)
memory usage: 220.0+ bytes


In [12]:
tqdm.pandas()

datos_fen = []


for idx, row in tqdm(df.iterrows(), total=len(df)):
    board = chess.Board()
    partida_id = idx
    jugadas = row['SAN']
    evaluaciones = row['evals']

    for i, jugada in enumerate(jugadas):
        try:
            board.push_san(jugada)
            eval_actual = evaluaciones[i] if i < len(evaluaciones) else None

            datos_fen.append({
                'ID_partida': partida_id,
                'jugada_num': i + 1,
                'SAN': jugada,
                'eval': eval_actual,  
                'turno': 'White' if board.turn == chess.BLACK else 'Black',  
                'FEN': board.fen(),
                'Resultado': row.get('Result', None),
                'White': row.get('White', None),
                'Black': row.get('Black', None),
                'WhiteElo': row.get('WhiteElo', None),
                'BlackElo': row.get('BlackElo', None),
                'ECO': row.get('ECO', None),
                'ECO_Family': row.get('ECO_Family', None),
                'TimeControl': row.get('TimeControl', None),
                'Termination': row.get('Termination', None),
            })
        except Exception:
            break  

df_fens = pd.DataFrame(datos_fen)

df_fens.to_csv('fens_por_jugada_con_evaluacion_input.csv', index=False)


100%|██████████| 1/1 [00:00<00:00, 249.91it/s]


In [13]:
df_fens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_partida   27 non-null     int64  
 1   jugada_num   27 non-null     int64  
 2   SAN          27 non-null     object 
 3   eval         26 non-null     float64
 4   turno        27 non-null     object 
 5   FEN          27 non-null     object 
 6   Resultado    27 non-null     int64  
 7   White        27 non-null     object 
 8   Black        27 non-null     object 
 9   WhiteElo     27 non-null     int64  
 10  BlackElo     27 non-null     int64  
 11  ECO          27 non-null     object 
 12  ECO_Family   27 non-null     object 
 13  TimeControl  27 non-null     object 
 14  Termination  27 non-null     object 
dtypes: float64(1), int64(5), object(9)
memory usage: 3.3+ KB


In [14]:
def clasificar_jugadas(df):
    calidad = []

    for idx, partida in df.groupby('ID_partida'):
        jugadas = partida.sort_values('jugada_num')
        prev_eval = None
        calidad_partida = []

        for eval_actual in jugadas['eval']:
            if prev_eval is None or pd.isna(prev_eval) or pd.isna(eval_actual):
                calidad_partida.append('desconocida')
            else:
                delta = eval_actual - prev_eval

                if delta >= -0.2:
                    calidad_partida.append('buena')
                elif delta >= -0.5:
                    calidad_partida.append('dudosa')
                else:
                    calidad_partida.append('mala')

            prev_eval = eval_actual

        calidad.extend(calidad_partida)

    df['calidad_jugada'] = calidad
    return df

df_fens = clasificar_jugadas(df_fens)


In [15]:
def extraer_features_fen(fen):
    board = chess.Board(fen)
    
    piezas = board.piece_map().values()
    material_blancas = sum(p.piece_type if p.color else 0 for p in piezas)
    material_negras = sum(p.piece_type if not p.color else 0 for p in piezas)
    
    return {
        'material_blancas': material_blancas,
        'material_negras': material_negras,
        'turno': int(board.turn),  # 1 si blancas, 0 si negras
        'enroque_blancas': int(board.has_kingside_castling_rights(chess.WHITE) or board.has_queenside_castling_rights(chess.WHITE)),
        'enroque_negras': int(board.has_kingside_castling_rights(chess.BLACK) or board.has_queenside_castling_rights(chess.BLACK)),
        'jaque': int(board.is_check()),
        'mate': int(board.is_checkmate())
    }


In [16]:
def extraer_features_fen(fen):
    board = chess.Board(fen)

    valor_piezas = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0}

    material_blancas = 0
    material_negras = 0

    for pieza in board.piece_map().values():
        valor = valor_piezas.get(pieza.piece_type, 0)
        if pieza.color == chess.WHITE:
            material_blancas += valor
        else:
            material_negras += valor

    return {
        'material_blancas': material_blancas,
        'material_negras': material_negras,
        'diferencia_material': material_blancas - material_negras,
        'turno': int(board.turn),  # 1 = blancas, 0 = negras
        'enroque_blancas': int(board.has_kingside_castling_rights(chess.WHITE) or board.has_queenside_castling_rights(chess.WHITE)),
        'enroque_negras': int(board.has_kingside_castling_rights(chess.BLACK) or board.has_queenside_castling_rights(chess.BLACK)),
        'jaque': int(board.is_check()),
        'mate': int(board.is_checkmate())
    }


In [17]:
tqdm.pandas() 

# Primero aplicamos la función para el cálculo de jugadas por material y luego lo añadimos al DF
features_df = df_fens['FEN'].progress_apply(extraer_features_fen).apply(pd.Series)
df_fens = pd.concat([df_fens, features_df], axis=1)


100%|██████████| 27/27 [00:00<00:00, 5400.13it/s]


In [18]:
df_fens.head()

,ID_partida,jugada_num,SAN,eval,turno,FEN,Resultado,White,Black,WhiteElo,...,Termination,calidad_jugada,material_blancas,material_negras,diferencia_material,turno,enroque_blancas,enroque_negras,jaque,mate
0,0,1,e4,0.34,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,1,Eduardo,Clase,200,...,normal,desconocida,39,39,0,0,1,1,0,0
1,0,2,b5,-0.24,Black,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,1,Eduardo,Clase,200,...,normal,mala,39,39,0,1,1,1,0,0
2,0,3,Bxb5,1.92,White,rnbqkbnr/p1pppppp/8/1B6/4P3/8/PPPP1PPP/RNBQK1N...,1,Eduardo,Clase,200,...,normal,buena,39,38,1,0,1,1,0,0
3,0,4,e6,-1.73,Black,rnbqkbnr/p1pp1ppp/4p3/1B6/4P3/8/PPPP1PPP/RNBQK...,1,Eduardo,Clase,200,...,normal,mala,39,38,1,1,1,1,0,0
4,0,5,Nf3,1.83,White,rnbqkbnr/p1pp1ppp/4p3/1B6/4P3/5N2/PPPP1PPP/RNB...,1,Eduardo,Clase,200,...,normal,buena,39,38,1,0,1,1,0,0


In [19]:
df_fens.to_csv('../data/raw/df_final_input.csv', index=False)

In [20]:
df_vector = pd.read_csv('../data/raw/df_final_input.csv')

In [21]:
piezas = {
    None: 0,
    chess.PAWN: 1,
    chess.KNIGHT: 2,
    chess.BISHOP: 3,
    chess.ROOK: 4,
    chess.QUEEN: 5,
    chess.KING: 6
}

# Función para vectorizar una posición FEN
def vectorizar_fen(fen):
    board = chess.Board(fen)
    vector = []
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            val = piezas[piece.piece_type]
            val = val if piece.color == chess.WHITE else -val
        else:
            val = 0
        vector.append(val)
    return vector  


tqdm.pandas()
vector_fens = df_vector['FEN'].progress_apply(vectorizar_fen)


fen_vector_df = pd.DataFrame(vector_fens.tolist(), columns=[f'FEN_{i}' for i in range(64)])


df_fens_vectorizado = pd.concat([df_fens.reset_index(drop=True), fen_vector_df], axis=1)

df_fens_vectorizado.to_csv('../data/raw/df_fens_vectorizado_input.csv', index=False)


100%|██████████| 27/27 [00:00<00:00, 6562.33it/s]


In [22]:
df_final = pd.read_csv('../data/raw/df_fens_vectorizado_input.csv')

df_final.head(3)

,ID_partida,jugada_num,SAN,eval,turno,FEN,Resultado,White,Black,WhiteElo,...,FEN_54,FEN_55,FEN_56,FEN_57,FEN_58,FEN_59,FEN_60,FEN_61,FEN_62,FEN_63
0,0,1,e4,0.34,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,1,Eduardo,Clase,200,...,-1,-1,-4,-2,-3,-5,-6,-3,-2,-4
1,0,2,b5,-0.24,Black,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,1,Eduardo,Clase,200,...,-1,-1,-4,-2,-3,-5,-6,-3,-2,-4
2,0,3,Bxb5,1.92,White,rnbqkbnr/p1pppppp/8/1B6/4P3/8/PPPP1PPP/RNBQK1N...,1,Eduardo,Clase,200,...,-1,-1,-4,-2,-3,-5,-6,-3,-2,-4


In [23]:
# Clasifica jugadas como buena, mala o dudosa en función del cambio de eval
def clasificar_jugada(eval_actual, eval_anterior):
    if pd.isna(eval_actual) or pd.isna(eval_anterior):
        return 'desconocida'
    delta = eval_actual - eval_anterior
    if abs(delta) < 0.15:
        return 'buena'
    elif delta < -0.15:
        return 'error'
    else:
        return 'dudosa'


df_final['eval_anterior'] = df_final.groupby('ID_partida')['eval'].shift(1)

if 'calidad_jugada' not in df_final.columns:
    df_final['calidad_jugada'] = df_final.apply(lambda row: clasificar_jugada(row['eval'], row['eval_anterior']), axis=1)

# Agrupación por partida para obtener los porcentajes
resumen = (
    df_final.groupby('ID_partida')['calidad_jugada']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .rename(columns={
        'buena': 'pct_buenas',
        'dudosa': 'pct_dudosas',
        'error': 'pct_errores'
    }))

# Asegurarse de que todas las columnas estén
for col in ['pct_buenas', 'pct_dudosas', 'pct_errores']:
    if col not in resumen.columns:
        resumen[col] = 0.0

df_ml = df_final.merge(resumen, on='ID_partida', how='left')

df_ml.head()

,ID_partida,jugada_num,SAN,eval,turno,FEN,Resultado,White,Black,WhiteElo,...,FEN_60,FEN_61,FEN_62,FEN_63,eval_anterior,pct_buenas,desconocida,mala,pct_dudosas,pct_errores
0,0,1,e4,0.34,White,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,1,Eduardo,Clase,200,...,-6,-3,-2,-4,NaN,0.444444,0.074074,0.481481,0.0,0.0
1,0,2,b5,-0.24,Black,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,1,Eduardo,Clase,200,...,-6,-3,-2,-4,0.34,0.444444,0.074074,0.481481,0.0,0.0
2,0,3,Bxb5,1.92,White,rnbqkbnr/p1pppppp/8/1B6/4P3/8/PPPP1PPP/RNBQK1N...,1,Eduardo,Clase,200,...,-6,-3,-2,-4,-0.24,0.444444,0.074074,0.481481,0.0,0.0
3,0,4,e6,-1.73,Black,rnbqkbnr/p1pp1ppp/4p3/1B6/4P3/8/PPPP1PPP/RNBQK...,1,Eduardo,Clase,200,...,-6,-3,-2,-4,1.92,0.444444,0.074074,0.481481,0.0,0.0
4,0,5,Nf3,1.83,White,rnbqkbnr/p1pp1ppp/4p3/1B6/4P3/5N2/PPPP1PPP/RNB...,1,Eduardo,Clase,200,...,-6,-3,-2,-4,-1.73,0.444444,0.074074,0.481481,0.0,0.0


In [24]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 94 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_partida           27 non-null     int64  
 1   jugada_num           27 non-null     int64  
 2   SAN                  27 non-null     object 
 3   eval                 26 non-null     float64
 4   turno                27 non-null     object 
 5   FEN                  27 non-null     object 
 6   Resultado            27 non-null     int64  
 7   White                27 non-null     object 
 8   Black                27 non-null     object 
 9   WhiteElo             27 non-null     int64  
 10  BlackElo             27 non-null     int64  
 11  ECO                  27 non-null     object 
 12  ECO_Family           27 non-null     object 
 13  TimeControl          27 non-null     object 
 14  Termination          27 non-null     object 
 15  calidad_jugada       27 non-null     objec

In [25]:
df_ml.drop(columns=['FEN'], inplace=True)

df_ml.to_csv('../data/raw/df_ml_input.csv', index=False)
